<a href="https://colab.research.google.com/github/JHyunjun/SuperResolution/blob/main/231025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created By unknown-user(Hyundai CTO Dacon)
# From 23.10.14

In [ ]:
# Unzip
#%cd /content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/train_hr
#!unzip -qq "/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/train_hr/hr.zip"

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)


In [ ]:
# Salt and pepper Denoising

def selective_median_filter(img, kernel_size=3, threshold1 = 0, threshold2 = 1):
    # Masks where the image has values close to 0 or 1
    mask_salt = img > (1.0 - threshold1) # white
    mask_pepper = img < threshold2 # black
    mask_combined = tf.math.logical_or(mask_salt, mask_pepper).numpy()

    # Convert image to numpy for processing
    img_numpy = img.numpy()

    # Apply median filter only where the mask is true
    for i in range(img_numpy.shape[2]):  # For each channel
        channel = img_numpy[:,:,i]
        channel_filtered = median_filter(channel, size=kernel_size)
        #channel_filtered = uniform_filter(channel, size=kernel_size)
        # Replace only the noisy pixels
        channel[mask_combined[:,:,0]] = channel_filtered[mask_combined[:,:,0]]
        img_numpy[:,:,i] = channel

    return tf.convert_to_tensor(img_numpy, dtype=tf.float32)

In [ ]:
# Importing

import tensorflow as tf
import os
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Lambda, Input, Add, Dense, Reshape, Flatten, UpSampling2D, Conv2D, LeakyReLU, BatchNormalization, Activation,Conv2DTranspose
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
import numpy as np
import scipy.ndimage as ndimage
from scipy.ndimage import median_filter, uniform_filter
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import random
import cv2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.callbacks import Callback
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import load_model


In [ ]:
# Data
directory_path_train_lr = "/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/train_lr"
directory_path_train_hr = "/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/train_hr"

In [ ]:
batch_size = 32

start_slicing = 0
end_slicing = 10000
image_slicing = end_slicing - start_slicing

train_split = int(0.97 * image_slicing)
val_split = image_slicing - train_split

lr_image_files = sorted([os.path.join(directory_path_train_lr, f) for f in os.listdir(directory_path_train_lr)])
hr_image_files = sorted([os.path.join(directory_path_train_hr, f) for f in os.listdir(directory_path_train_hr)])

lr_image_files_3000 = lr_image_files[start_slicing:start_slicing + train_split]
hr_image_files_3000 = hr_image_files[start_slicing:start_slicing + train_split]
val_lr_files = lr_image_files[start_slicing + train_split:end_slicing]
val_hr_files = hr_image_files[start_slicing + train_split:end_slicing]

# pre-processing
def load_images(lr_path, hr_path):
    lr = tf.io.read_file(lr_path)
    lr = tf.image.decode_jpeg(lr, channels=3)
    lr = tf.image.resize(lr, [256, 256])
    lr = lr / 255.0
    lr_filter = tf.py_function(func=selective_median_filter, inp=[lr], Tout=tf.float32)

    hr = tf.io.read_file(hr_path)
    hr = tf.image.decode_jpeg(hr, channels=3)
    hr = tf.image.resize(hr, [1024, 1024])
    hr = hr / 255.0

    return lr_filter, hr
    #return lr, hr, lr_filter

train_dataset = tf.data.Dataset.from_tensor_slices((lr_image_files_3000, hr_image_files_3000))
train_dataset = train_dataset.map(load_images)
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_lr_files, val_hr_files))
val_dataset = val_dataset.map(load_images)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
def count_files_in_directory(directory_path):
    try:
        # 디렉토리에 있는 파일 리스트 가져오기
        files = os.listdir(directory_path)

        # 이미지 파일만 골라내기 (예: jpg, png 파일)
        image_files = [file for file in files if file.endswith(('.jpg', '.png'))]

        # 이미지 파일의 수 반환
        return len(image_files)

    except Exception as e:
        # 오류 발생 시 메시지 출력
        print(f"An error occurred: {e}")
        return None
print(f"Number of images in {directory_path_train_lr}: {count_files_in_directory(directory_path_train_lr)}")
print(f"Number of images in {directory_path_train_hr}: {count_files_in_directory(directory_path_train_hr)}")

In [ ]:
def show_images(dataset, num_samples=3):
    #for i, (lr_image, hr_image, lr_filter) in enumerate(dataset.unbatch().take(num_samples)):
    for i, (lr_image, hr_image) in enumerate(dataset.unbatch().take(num_samples)):
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 3, 1)
        plt.imshow(lr_image.numpy())
        plt.title(f"Low Resolution Image {i+1}")

        plt.subplot(1, 3, 2)
        #plt.imshow(lr_filter.numpy())
        plt.title(f"Filtering Image {i+1}")

        plt.subplot(1, 3, 3)
        plt.imshow(hr_image.numpy())
        plt.title(f"High Resolution Image {i+1}")

        plt.show()

show_images(val_dataset)

In [ ]:
def show_images(dataset, num_samples=3):
    #for i, (lr_image, hr_image, lr_filter) in enumerate(dataset.unbatch().take(num_samples)):
    for i, (lr_image, hr_image) in enumerate(dataset.unbatch().take(num_samples)):
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 3, 1)
        plt.imshow(lr_image.numpy())
        plt.title(f"Low Resolution Image {i+1}")

        plt.subplot(1, 3, 2)
        #plt.imshow(lr_filter.numpy())
        plt.title(f"Filtering Image {i+1}")

        plt.subplot(1, 3, 3)
        plt.imshow(hr_image.numpy())
        plt.title(f"High Resolution Image {i+1}")

        plt.show()

show_images(train_dataset)

In [ ]:
#Super Resolution

def combined_loss(alpha):
    mse_loss = MeanSquaredError()
    mae_loss = MeanAbsoluteError()

    def loss_function(y_true, y_pred):
        # Calculate MSE
        mse = mse_loss(y_true, y_pred)
        mae = mae_loss(y_true, y_pred)

        # Calculate PSNR
        max_pixel = 1.0
        psnr = 1.0 * (-10.0 * tf.math.log(max_pixel**2 / (mse + 1e-10)) / tf.math.log(10.0))

        # Combine the MSE and PSNR with a weight
        # combined_loss = alpha * mse + (1 - alpha) * psnr
        # combined_loss = alpha * mae + (1 - alpha) * psnr
        combined_loss = mae + mse

        return combined_loss

    return loss_function

def psnr_loss(y_true, y_pred):
    # Assuming the images are scaled between 0 and 1
    max_pixel = 1.0
    # Compute MSE
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    # Compute PSNR loss
    return -10 * tf.math.log(max_pixel**2 / (mse + 1e-10)) / tf.math.log(10.0)

def residual_block(x, filters, kernel_size=3, scaling=0.1):
    tmp = Conv2D(filters, kernel_size, padding='same', activation='relu')(x)
    tmp = Conv2D(filters, kernel_size, padding='same')(tmp)
    tmp = Add()([x, tmp])
    return tmp

def upsample(x, scale=4):
    return Lambda(lambda x: tf.nn.depth_to_space(x, scale))(x)

def EDSR(input_shape=(256,256,3), scale=4):
    inputs = Input(input_shape)

    x = Conv2D(64, 3, padding='same', activation='relu')(inputs)
    for _ in range(9):  # Number of residual blocks
        x = residual_block(x, 64)

    x = Conv2D(64, 3, padding='same')(x)
    x = upsample(x, scale)
    outputs = Conv2D(3, 3, padding='same')(x)

    return Model(inputs, outputs)

model = EDSR()
optimizer = Adam(learning_rate = 1e-3)
#model.compile(optimizer=optimizer, loss = 'mean_absolute_error')
model.compile(optimizer=optimizer, loss=combined_loss(alpha=1))
#model.compile(optimizer=optimizer, loss = 'mse')
model.summary()


In [ ]:
class PSNRCallback(Callback):
    def __init__(self, val_dataset):
        super().__init__()
        self.val_dataset = val_dataset

    def on_epoch_end(self, epoch, logs=None):
        total_psnr = 0
        num_batches = 0

        for x_val, y_val in self.val_dataset:
            y_pred = self.model.predict(x_val)

            # Assuming the images are scaled between 0 and 1
            max_pixel = 1.0
            mse = tf.reduce_mean(tf.square(y_val - y_pred))
            psnr = -10.0 * tf.math.log(mse + 1e-10) / tf.math.log(10.0)

            total_psnr += psnr
            num_batches += 1

        average_psnr = total_psnr / num_batches
        logs["val_psnr"] = average_psnr
        print(f"\nValidation PSNR after epoch {epoch+1}: {average_psnr:.4f} dB")

class CustomLRScheduler(Callback):
    def __init__(self, lr_decay_factor, decay_interval=10):
        super().__init__()
        self.lr_decay_factor = lr_decay_factor
        self.decay_interval = decay_interval

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.decay_interval == 0:
            current_lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
            new_lr = current_lr * self.lr_decay_factor
            tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)
            print(f"\nSetting learning rate to {new_lr}")

lr_decay_factor = 0.5
custom_lr_scheduler = CustomLRScheduler(lr_decay_factor=lr_decay_factor)

# Usage in model.fit()
psnr_callback = PSNRCallback(val_dataset=val_dataset)

In [ ]:
#Training

#early_stopping = EarlyStopping(monitor='val_psnr', patience=5, verbose=1, restore_best_weights=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)
history = model.fit(train_dataset, epochs=5, validation_data=val_dataset, callbacks=[psnr_callback, early_stopping, reduce_lr])

In [ ]:
model_s = load_model('/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test_hr/231024/code/4.h5', custom_objects={'loss_function': combined_loss(alpha=1)})
model_s.summary()

In [ ]:
# New Training
new_learning_rate = 1e-5
optimizer = Adam(learning_rate=new_learning_rate)
model_s.compile(optimizer=optimizer, loss=combined_loss(alpha=1))
reduce_lr2 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_lr=1e-6)
history = model_s.fit(train_dataset, epochs=3, validation_data=val_dataset, callbacks=[psnr_callback, early_stopping, reduce_lr2])

In [ ]:
model_s.save('/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test_hr/231024/code/1.h5')

In [ ]:
model_s2 = load_model('/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test_hr/231024/code/1.h5', custom_objects={'loss_function': combined_loss(alpha=1)})

In [ ]:
# New Training
new_learning_rate = 1e-4
optimizer = Adam(learning_rate=new_learning_rate)
model_s2.compile(optimizer=optimizer, loss=combined_loss(alpha=1))
reduce_lr2 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)
history = model_s2.fit(train_dataset, epochs=5, validation_data=val_dataset, callbacks=[psnr_callback, early_stopping, reduce_lr2])

In [ ]:
def show_images(dataset, num_samples=3):
    #for i, (lr_image, hr_image, lr_filter) in enumerate(dataset.unbatch().take(num_samples)):
    for i, (lr_image, hr_image) in enumerate(dataset.unbatch().take(num_samples)):
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 3, 1)
        plt.imshow(lr_image.numpy())
        plt.title(f"Low Resolution Image {i+1}")

        lr_image = tf.expand_dims(lr_image, 0)
        sr = model_s2.predict(lr_image)
        sr = np.clip(sr, 0, 1)
        plt.subplot(1, 3, 2)
        plt.imshow(tf.squeeze(sr))
        plt.title(f"Super Resolution Image {i+1}")

        plt.subplot(1, 3, 3)
        plt.imshow(hr_image.numpy())
        plt.title(f"High Resolution Image {i+1}")

        plt.show()

show_images(train_dataset)

In [ ]:
model_s2.save('/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test_hr/231024/code/2.h5')
model_s3 = load_model('/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test_hr/231024/code/2.h5', custom_objects={'loss_function': combined_loss(alpha=1)})

In [ ]:
# New Training
new_learning_rate = 1e-3
optimizer = Adam(learning_rate=new_learning_rate)
model_s3.compile(optimizer=optimizer, loss=combined_loss(alpha=1))
reduce_lr2 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_lr=1e-6)
history = model_s3.fit(train_dataset, epochs=3, validation_data=val_dataset, callbacks=[psnr_callback, early_stopping, reduce_lr2])

In [ ]:
model_s3.save('/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test_hr/231023/code/3.h5')
model_s4 = load_model('/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test_hr/231023/code/3.h5', custom_objects={'loss_function': combined_loss(alpha=1)})

In [ ]:
# New Training
new_learning_rate = 1e-5
optimizer = Adam(learning_rate=new_learning_rate)
model_s4.compile(optimizer=optimizer, loss=combined_loss(alpha=1))
reduce_lr2 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)
history = model_s4.fit(train_dataset, epochs=2, validation_data=val_dataset, callbacks=[psnr_callback, early_stopping, reduce_lr2])

In [ ]:
model_s4.save('/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test_hr/231024/code/4.h5')
model_s5 = load_model('/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test_hr/231024/code/4.h5', custom_objects={'loss_function': combined_loss(alpha=1)})

In [ ]:
# New Training
new_learning_rate = 5e-4
optimizer = Adam(learning_rate=new_learning_rate)
model_s5.compile(optimizer=optimizer, loss=combined_loss(alpha=1))
reduce_lr2 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1, min_lr=1e-6)
history = model_s5.fit(train_dataset, epochs=30, validation_data=val_dataset, callbacks=[psnr_callback, early_stopping, reduce_lr2])

In [ ]:
model_s5.save('/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test_hr/231023/code/5.h5')
model_s6 = load_model('/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test_hr/231023/code/5.h5', custom_objects={'loss_function': combined_loss(alpha=1)})

In [ ]:
# New Training
new_learning_rate = 5e-4
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
optimizer = Adam(learning_rate=new_learning_rate)
model_s6.compile(optimizer=optimizer, loss=combined_loss(alpha=1))
reduce_lr2 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1, min_lr=1e-6)
history = model_s6.fit(train_dataset, epochs=5, validation_data=val_dataset, callbacks=[psnr_callback, early_stopping, reduce_lr2])

In [ ]:
def show_images(dataset, num_samples=3):
    #for i, (lr_image, hr_image, lr_filter) in enumerate(dataset.unbatch().take(num_samples)):
    for i, (lr_image, hr_image) in enumerate(dataset.unbatch().take(num_samples)):
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 3, 1)
        plt.imshow(lr_image.numpy())
        plt.title(f"Low Resolution Image {i+1}")

        lr_image = tf.expand_dims(lr_image, 0)
        sr = model_s6.predict(lr_image)
        sr = np.clip(sr, 0, 1)
        plt.subplot(1, 3, 2)
        plt.imshow(tf.squeeze(sr))
        plt.title(f"Super Resolution Image {i+1}")

        plt.subplot(1, 3, 3)
        plt.imshow(hr_image.numpy())
        plt.title(f"High Resolution Image {i+1}")

        plt.show()

show_images(train_dataset)

In [ ]:
# Testifying

directory_path_test_lr = "/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test/lr"
test_image_files = sorted([os.path.join(directory_path_test_lr, fname) for fname in os.listdir(directory_path_test_lr) if fname.endswith('.png') or fname.endswith('.jpg')])

def load_and_preprocess_image(img_path):
    img = load_img(img_path, target_size=(256, 256))
    img = img_to_array(img)
    img = img / 255.0
    img_filter = tf.py_function(func=selective_median_filter, inp=[img], Tout=tf.float32)
    return img_filter

selected_test_image_files = random.sample(test_image_files, 10)

for test_img_path in selected_test_image_files:
    test_img = load_and_preprocess_image(test_img_path)
    test_img_input = tf.expand_dims(test_img, 0)

    sr_img = model_s3.predict(test_img_input)
    sr_img = tf.squeeze(sr_img)
    sr_img = np.clip(sr_img, 0, 1)

    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(test_img)
    plt.title("LR Image")

    plt.subplot(1, 2, 2)
    plt.imshow(sr_img)
    plt.title("SR Image")

    plt.show()

In [ ]:
# Data saving for submission-1

directory_path_test_lr = "/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test/lr"
test_image_files = [os.path.join(directory_path_test_lr, fname) for fname in os.listdir(directory_path_test_lr) if fname.endswith('.png') or fname.endswith('.jpg')]

def load_and_preprocess_image(img_path):
    img = load_img(img_path, target_size=(256, 256))
    img = img_to_array(img)
    img = img / 255.0
    img_filter = tf.py_function(func=selective_median_filter, inp=[img], Tout=tf.float32)
    return img_filter

save_directory = "/content/drive/MyDrive/Colab_Notebooks/Hyundai_Daycon/test_hr/231024/2"

for test_img_path in test_image_files:
    test_img = load_and_preprocess_image(test_img_path)
    test_img_input = tf.expand_dims(test_img, 0)

    sr_img = model_s5.predict(test_img_input)
    sr_img = tf.squeeze(sr_img).numpy()
    sr_img = np.clip(sr_img * 255, 0, 255).astype(np.uint8)  #De-normalizing
    # RGB to BGR
    sr_img_bgr = cv2.cvtColor(sr_img, cv2.COLOR_RGB2BGR)

    # Use the same filename from input
    filename = os.path.join(save_directory, os.path.basename(test_img_path))

    cv2.imwrite(filename, sr_img_bgr)


In [ ]:
# Model save

model_path = os.path.join(save_directory, '231017_model_117.h5')
model.save(model_path)

In [ ]:
# Additional Super Resolution Algorithm

def generate_sr_images(lr_images, hr_images):
    sr_images = model(lr_images)
    return sr_images, hr_images

train_dataset_sr = train_dataset.map(generate_sr_images)
val_dataset_sr = val_dataset.map(generate_sr_images)

second_model = Sequential([
    Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(1024, 1024, 3)),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    Conv2D(3,(3, 3), padding='same', activation='relu')
])
second_model.compile(optimizer='adam', loss='mse')
second_model.summary()

In [ ]:
# Second training
second_model.fit(train_dataset_sr, epochs=3, validation_data=val_dataset_sr)

In [ ]:
# Visualization

sample_lr_images, sample_hr_images = next(iter(train_dataset.take(3)))

first_model_outputs = model.predict(sample_lr_images)
second_model_outputs = second_model.predict(first_model_outputs)

sr_img = tf.squeeze(sr_img)
sr_img = np.clip(sr_img, 0, 1)

fig, axs = plt.subplots(10, 4, figsize=(15, 30))

for i in range(10):
    axs[i, 0].imshow(sample_lr_images[i].numpy())
    axs[i, 0].set_title('LR Image')
    axs[i, 0].axis('off')

    axs[i, 1].imshow(first_model_outputs[i])
    axs[i, 1].set_title('First Model Output')
    axs[i, 1].axis('off')

    axs[i, 2].imshow(second_model_outputs[i])
    axs[i, 2].set_title('Second Model Output')
    axs[i, 2].axis('off')

    axs[i, 3].imshow(sample_hr_images[i].numpy())
    axs[i, 3].set_title('HR Image')
    axs[i, 3].axis('off')

plt.tight_layout()
plt.show()